In [8]:
# normalized VGG feature data that has been smoted and sent through simple LSTM


# First, import necessary libraries
import tensorflow as tf
import numpy as np
import keras.backend as K

In [9]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

In [27]:
# loading the smoted training data
X_smote = np.load('../Desktop/X_smote.npy')
Y_smote = np.load('../Desktop/Y_smote.npy')

X_smote_train = X_smote[:4, :, :]
Y_smote_train = Y_smote[:4, :]
X_smote_test = X_smote[4:, :, :]
Y_smote_test = Y_smote[4:, :]

In [ ]:
# important vars
timesteps = X_smote_train.shape[1]
data_dim = X_smote_train.shape[2]

In [19]:
X_reg = np.load('../Desktop/VGG16_feature_data.npy')
Y_reg = np.load('../Desktop/labels.npy')

X_test = np.zeros([3, timesteps, data_dim])
Y_test = np.zeros([3, timesteps])

X_test[:, :X_reg.shape[1], :] = X_reg[4:, :, :]
Y_test[:, :X_reg.shape[1]] = Y_reg[4:, :212]

print(X_test.shape)
print(Y_test.shape)

(3, 420, 4096)
(3, 420)


In [20]:
# constructing a many-to-one LSTM model in keras --> inspiration: https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras
# i will start by training a model on only the VGG16 fc6 layer output (that's just one feature)
# should I eventually abstract this LSTM model? Create its own object file?
model = Sequential()

# normalization
model.add(BatchNormalization(input_shape=(timesteps, data_dim)))

model.add(LSTM(timesteps, return_sequences=True))
#input_shape=(timesteps, data_dim)

# going to try adding a flatten layer in here
model.add(Flatten()) # I got this from a github thing, but I still don't completely understand why it works
# add the final dense layer and then softmax
model.add(Dense(timesteps, activation='sigmoid'))
# going to add a softmax activation to this
#model.add(Activation('softmax'))

In [21]:
# a new metric of evaluation! the F-score!
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [28]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy', FScore2])

In [29]:
# running the LSTM model
model.fit(X_smote_train, Y_smote_train, epochs = 20, validation_data=(X_smote_test, Y_smote_test))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/20
4/4 [==============================] - 30s 7s/step - loss: 4.6563e-07 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 4.6424 - val_binary_accuracy: 0.6810 - val_FScore2: 0.5867
Epoch 2/20
4/4 [==============================] - 20s 5s/step - loss: 1.0916e-07 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 4.6423 - val_binary_accuracy: 0.6810 - val_FScore2: 0.5867
Epoch 3/20
4/4 [==============================] - 20s 5s/step - loss: 1.0904e-07 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 4.6422 - val_binary_accuracy: 0.6810 - val_FScore2: 0.5867
Epoch 4/20
4/4 [==============================] - 18s 5s/step - loss: 1.0896e-07 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 4.6422 - val_binary_accuracy: 0.6810 - val_FScore2: 0.5867
Epoch 5/20
4/4 [==============================] - 17s 4s/step - loss: 1.0891e-07 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 4.6421 - val_binary_accuracy: 0.6810 

In [ ]:
# trying to view the model output
out = model.predict(X_train_data)
print("model prediction:")
print(out[0])
print("target:")
print(Y_train_data[0])

print("before 64:")
print(out[0][63])
print("64:")
print(out[0][65])

print("rounded")
print(np.round(out)[0])

In [ ]:
#try visualizing this model at some point?